# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Passo 1) Pesquisar as cotações das moedas e ouro;
# Passo 2) Buscar as informações no site;
# Passo 3) Armazenar as informações;


navegador = webdriver.Chrome()
    #Passos para cotação de dolar
    #Passo1: Entrar no site que estamos buscando
navegador.get("https://www.google.com.br/")
    #Passo2: No campo de pesquisas digita "cotação de dólar"
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
    #Passo3: Aperta Enter
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    #Passo4: Coleta o valor da cotação
cotacao_dolar = navegador.find_element(By.XPATH, 
'/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação do dólar: ",cotacao_dolar)

    #Cotação do Euro
navegador.get("https://www.google.com.br/")
    #Passo2: No campo de pesquisas digita "cotação de euro"
navegador.find_element(By.XPATH, 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
    #Passo3: Aperta Enter
navegador.find_element(By.XPATH, 
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    #Passo4: Coleta o valor da cotação
cotacao_euro = navegador.find_element(By.XPATH, 
'/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação do euro: ",cotacao_euro)

    #Cotação do ouro
    #Passo1: Entrar no site que estamos buscando
navegador.get("https://www.melhorcambio.com/ouro-hoje")
    #Passo2: Coleta do valor da cotação
cotacao_ouro = navegador.find_element(By.XPATH, '/html/body/div[6]/div[1]/div/div/input[2]').get_attribute("value")
print("Cotação do ouro: ", cotacao_ouro)

    




Cotação do dólar:  5.012099999999999
Cotação do euro:  5.507245359000001
Cotação do ouro:  324,63


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
import pandas as pd
# Passo 4) Preencher a cotação em uma planilha do excel;
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_dolar)


tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.012100,5012.049879,1.40,7016.869831
1,Carro Renault,4500.00,Euro,5.507245,24782.604116,2.00,49565.208231
2,Notebook Dell,899.99,Dólar,5.012100,4510.839879,1.70,7668.427794
3,IPhone,799.00,Dólar,5.012100,4004.667900,1.70,6807.935430
4,Carro Fiat,3000.00,Euro,5.507245,16521.736077,1.90,31391.298546
5,Celular Xiaomi,480.48,Dólar,5.012100,2408.213808,2.00,4816.427616
6,Joia 20g,20.00,Ouro,5.012100,100.242000,1.15,115.278300


### Agora vamos exportar a nova base de preços atualizada

In [4]:
tabela.to_excel("Produtos Novo.xlsx", index=False)